In [ ]:
import base64
import io
import numpy as np
import matplotlib.image as mpimg

from imageio import imread
import matplotlib.pyplot as plt

In [ ]:
encodedd = base64.b64encode(img.read())
decoded =base64.decodebytes(encodedd)

In [ ]:
def Resize(mat,new_x,new_y):
    x,y,z =np.shape(mat)
    RR = x/new_x
    CR = y/new_y
    newmat = np.zeros((new_x,new_y,z))
    for i in range(1,new_x+1):
        old_x = round(i*RR)
        if old_x == 0:
            old_x = 1
        for j in range(1,new_y+1):
            old_y = round(j*CR)
            if old_y == 0:
                old_y = 1
            for k in range(z):
                newmat[i-1][j-1][k]=mat[old_x-1][old_y-1][k]
    return np.uint(newmat) 

In [ ]:
def grayImageAvg(mat):
    if len(mat.shape)==2:
        return mat
    R = mat[:,:,0]
    G = mat[:,:,1]
    B = mat[:,:,2]
    gray = (1/3) *R + (1/3) * G + (1/3) * B
    return np.uint8(gray)

In [ ]:
def grayImage2(mat):
    if len(mat.shape)==2:
        return mat
    R = mat[:,:,0]
    G = mat[:,:,1]
    B = mat[:,:,2]
    gray = 0.2989 * R + 0.5870 * G + 0.1140 * B
    return np.uint8(gray)

In [ ]:
def grayImageDec(mat):
    if len(mat.shape)==2:
        return mat
    R = mat[:,:,0]
    G = mat[:,:,1]
    B = mat[:,:,2]
    gray = min(R,G,B)
    return np.uint8(gray)

In [ ]:
def hist(mat):
    graymat = grayImage2(mat);
    counter = np.zeros(255)
    x,y= graymat.shape
    for i in range(x):
            for j in range(y):
                counter[int(graymat[i,j])] = counter[int(graymat[i,j])]+1
    return counter
fig = plt.figure()
#fig.bar(range(255), hist(img))
#fig.add_subplot(plt.plot(hist(img)))
plt.plot(hist(img))
#fig = plt.gcf()

#plt.savefig('foo.pdf')

In [ ]:
def Contrast(mat,newMin=0,newMax=255):
    graymat = grayImage2(mat)
    x,y= graymat.shape
    res=np.zeros(shape=(x,y))
    oldMin=graymat[:,:].min()
    oldMax=graymat[:,:].max()
    res = ( ( (graymat - oldMin) / (oldMax - oldMin) ) * (newMax - newMin) ) + newMin;
    res=np.clip(res,0,255)
    return res 

In [ ]:
def bright(mat,off):
    x,y,_= mat.shape
    res=np.array(mat+off)  
    res=np.clip(res,0,255)
    return res
img=bright(img,50)
plt.imshow(img)

In [ ]:
def powerLow(mat,G):
    res=np.power(mat,G)
    res = Contrast(res,0,255)
    return res

In [ ]:
def add(m,n):
    m = grayImage2(m)
    n = grayImage2(n)
    x1,y1 = m.shape
    x2,y2 = n.shape
    max_x = max(x1,x2)
    max_y = max(y1,y2)
    min_x = min(x1,x2)
    min_y = min(y1,y2)
    res = np.zeros ((max_x,max_y))
    if x1>=x2:
        res[x2:x1,:min_y] = m[x2:x1,:min_y]
    else:
        res[x1:x2,:min_y] = n[x1:x2,:min_y]
    if y1>=y2:
        res[:min_x,y2:y1] = m[:min_x,y2:y1]
    else:
        res[:min_x,y1:y2] = n[:min_x,y1:y2]
    res[:min_x,:min_y] = m[:min_x,:min_y] + n[:min_x,:min_y]
    res[-1,-1] = np.mean([m[-1,-1],n[-1,-1]])
    res
    res=np.clip(res,0,255)
    return res

In [ ]:
x1,y1 = m.shape[:2]
x2,y2 = n.shape[:2]
max_x = max(x1,x2)
max_y = max(y1,y2)
min_x = min(x1,x2)
min_y = min(y1,y2)
res = np.zeros ((max_x,max_y))
m =np.pad(m, (max_x-x1,max_y-y1),'constant',constant_values=(0,0))
n =np.pad(n, (max_x-x2,max_y-y2), 'constant',constant_values=(0,0))
res=m+n

In [ ]:
def sub(mat1,mat2):
    res = mat1-mat2
    res=np.clip(res,0,255)
    return res

In [ ]:
def negative(mat1):
    res = 255-mat1
    return res

In [ ]:
def Qunt(mat,k):
    mat=grayImage2(mat)
    if(k==8):
        return mat
    grayLevel = 2**k
    g = 256/(grayLevel)
    g2 = 255/(grayLevel-1)
    temp = mat/g
    indx = np.floor(temp)
    newimg = indx*g2
    return np.uint8(newimg)

In [ ]:
def smoothAvgZeroPaddF(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    temp = []
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)),'constant',constant_values=(0,0))
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                newimg[i,j] +=np.sum(img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1])
                newimg[i,j] *=1/filterP2
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def smoothAvgRepPaddF(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                newimg[i,j] +=np.sum(img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1])
                newimg[i,j] *=1/filterP2
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def smoothMin(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    temp = []
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                temp = img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1]
                newimg[i,j] = np.min(temp)
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def smoothMax(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    temp = []
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                temp = img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1]
                newimg[i,j] = np.max(temp)
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def smoothMedian(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    temp = []
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                temp = img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1]
                newimg[i,j] = np.median(temp)
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def smoothMidPoint(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    temp = []
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                temp = img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1]
                newimg[i,j] = (np.max(temp)/2 + np.min(temp)/2)
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def smoothGauss(mat,sigma):
    n = int(3.7*sigma -0.5)
    filter_size = 2*n +1
    mat = grayImage2(mat)
    xo,yo = mat.shape
    filterHalf=int(np.floor(filter_size/2))
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    x,y = img.shape
    newimg=np.zeros([x,y])
    newimg=np.int32(newimg)
    Filter = np.zeros((filter_size,filter_size))
    for i in range(filter_size):
        for j in range(filter_size):
            xfilter = np.linspace(-(filter_size // 2),filter_size // 2, filter_size)
            yfilter = np.linspace(-(filter_size // 2),filter_size // 2, filter_size)
            Filter[i,j] = (1/(2*np.pi*sigma**2)) * np.exp(-1*(xfilter[i]**2+yfilter[j]**2)/(2*sigma**2))
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                newimg[i,j] = np.sum(np.multiply(img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1],Filter))
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
from sklearn import preprocessing
def edgeDetection(mat):
    mat = grayImage2(mat)
    x, y = mat.shape
    newimg=np.full((x,y),255)
    newimg=np.int32(newimg)
    for i in range(x):
        for j in range(y):
            if(i==0 or j==0 or i==x-1 or j==y-1):
                newimg[i,j] = mat[i,j]
            else:
                newimg[i ,j] =(-1*mat[i+1,j]-1*mat[i-1,j]- 1*mat[i,j+1] - 1*mat[i,j-1] +4*mat[i,j])           
    newimg = newimg/255
    newimg = preprocessing.normalize(newimg)
    newimg*=255
    #newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def Sharp(mat,sharping_rate=1):
    mat = grayImage2(mat)
    x,y = mat.shape
    newimg=np.full((x,y),0)
    newimg=np.int32(newimg)
    for i in range(1,x-1):
        for j in range(1,y-1):
            if(i==0 or j==0 or i==x-1 or j==y-1):
                newimg[i,j] = mat[i,j]
            else:
                res = sharping_rate*(-1*mat[i+1,j]-mat[i-1,j]-mat[i,j+1] - mat[i,j-1] +4*mat[i,j])
                newimg[i ,j] = mat[i,j]+ res
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)

In [ ]:
def unSharpe(mat):
    newimg = mat + smoothGauss(mat,2)
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)    

In [ ]:
def lowIdealFilter(mat,D0=1):
    img = grayImage2(mat)
    x,y = img.shape
    FT_img = np.fft.fft2(img)
    newimg = np.zeros((x,y))
    u = np.arange(0,x)
    idx = np.where( u > x/2)
    u[idx] = u[idx]-x
    v = np.arange(0,y)
    idy = np.where( v > y/2)
    v[idy] = v[idy]-y
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2+V**2);
    H = (D <= D0)
    G = H*FT_img
    newimg = np.real(np.fft.ifft2(G))
    return newimg

In [ ]:
def lowButterworthFilter(mat,D0=1,n=1):
    img = grayImage2(mat)
    x,y = img.shape
    FT_img = np.fft.fft2(img)
    newimg = np.zeros((x,y))
    u = np.arange(0,x)
    idx = np.where( u > x/2)
    u[idx] = u[idx]-x
    v = np.arange(0,y)
    idy = np.where( v > y/2)
    v[idy] = v[idy]-y
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2+V**2);
    H = 1 / (1+(D/D0)**(2*n))
    G = H*FT_img
    newimg = np.real(np.fft.ifft2(G))
    return newimg

In [ ]:
def lowGaussianFilter(mat,D0=1):
    img = grayImage2(mat)
    x,y = img.shape
    FT_img = np.fft.fft2(img)
    newimg = np.zeros((x,y))
    u = np.arange(0,x)
    idx = np.where( u > x/2)
    u[idx] = u[idx]-x
    v = np.arange(0,y)
    idy = np.where( v > y/2)
    v[idy] = v[idy]-y
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2+V**2);
    H = np.exp(-(D**2)/(2*(D0**2)))
    G = H * FT_img
    newimg = np.real(np.fft.ifft2(G))
    return newimg

In [ ]:
def highIdealFilter(mat,D0=1):
    img = grayImage2(mat)
    x,y = img.shape
    FT_img = np.fft.fft2(img)
    newimg = np.zeros((x,y))
    u = np.arange(0,x)
    idx = np.where( u > x/2)
    u[idx] = u[idx]-x
    v = np.arange(0,y)
    idy = np.where( v > y/2)
    v[idy] = v[idy]-y
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2+V**2);
    H = (D <= D0)
    H = 1-H
    G = H*FT_img
    newimg = np.real(np.fft.ifft2(G))
    return newimg

In [ ]:
def highButterworthFilter(mat,D0=1,n=1):
    img = grayImage2(mat)
    x,y = img.shape
    FT_img = np.fft.fft2(img)
    newimg = np.zeros((x,y))
    u = np.arange(0,x)
    idx = np.where( u > x/2)
    u[idx] = u[idx]-x
    v = np.arange(0,y)
    idy = np.where( v > y/2)
    v[idy] = v[idy]-y
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2+V**2);
    H = 1 / (1+(D0/D)**(2*n))
    G = H*FT_img
    newimg = np.real(np.fft.ifft2(G))
    return newimg

In [ ]:
def highGaussianFilter(mat,D0=1):
    img = grayImage2(mat)
    x,y = img.shape
    FT_img = np.fft.fft2(img)
    newimg = np.zeros((x,y))
    u = np.arange(0,x)
    idx = np.where( u > x/2)
    u[idx] = u[idx]-x
    v = np.arange(0,y)
    idy = np.where( v > y/2)
    v[idy] = v[idy]-y
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2+V**2);
    H = 1-np.exp(-(D**2)/(2*(D0**2)))
    G = H * FT_img
    newimg = np.real(np.fft.ifft2(G))
    return newimg

In [ ]:
def saltPaper(img,max=255):
    img=np.uint32(img)
    saltPaperImg=np.random.randint(max+1,size=img.shape)
    saltPaperImg[(saltPaperImg>0) &  (saltPaperImg<max)]=1
    saltPaperImg[(saltPaperImg==max)]=255
    new=np.zeros(img.shape)
    new=np.uint32(new)
    new=np.multiply(img,saltPaperImg)
    new=np.clip(new,0,255)
    return new

In [ ]:
def noisy(image):
    row,col= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = image + gauss
    return noisy
new = noisy(img)

In [ ]:
from skimage.util import random_noise

In [ ]:
def geoMeanFilter(mat,filter_size):
    filterHalf=np.floor(filter_size/2)
    mat = grayImage2(mat)
    xo,yo = mat.shape
    img =np.pad(mat, (int(filter_size/2),int(filter_size/2)), mode='edge')
    img=np.uint32(img)
    x,y = img.shape
    filterP2=filter_size**2
    newimg=np.ones([x,y],np.uint64)
    for i in range(x):
        print(i)
        for j in range(y):
            if(i-x+filterHalf>=0 or j-y+filterHalf>=0 or i<filterHalf or j<filterHalf):
                newimg[i,j] = img[i,j]
            else:
                newimg[i, j] = np.prod(img[i-int(filterHalf):i+int(filterHalf)+1,j-int(filterHalf):j+int(filterHalf)+1]**(1/(filter_size**2)))
    newimg=np.clip(newimg,0,255)
    return np.uint8(newimg)
plt.figure()
plt.imshow(geoMeanFilter(img,30),cmap='gray')